# Challenge Set 9
## Part III: Soccer Data

*Introductory - Intermediate level SQL*

---

Please complete this exercise using sqlite3 and Jupyter notebook.

Download the [SQLite database](https://www.kaggle.com/hugomathien/soccer/downloads/soccer.zip) and load in your notebook using the sqlite3 library. 

1. Which team scored the most points when playing at home?  

2. Did this team also score the most points when playing away?  

3. How many matches resulted in a tie?  

4. How many players have Smith for their last name? How many have 'smith' anywhere in their name?

5. What was the median tie score? Use the value determined in the previous question for the number of tie games. *Hint:* PostgreSQL does not have a median function. Instead, think about the steps required to calculate a median and use the [`WITH`](https://www.postgresql.org/docs/8.4/static/queries-with.html) command to store stepwise results as a table and then operate on these results. 

6. What percentage of players prefer their left or right foot? *Hint:* Calculate either the right or left foot, whichever is easier based on how you setup the problem.

## Finding Tables

In [1]:
import sqlite3
import pandas as pd

In [2]:
cnx = sqlite3.connect("./metis-challenges-master/data/database.sqlite")

In [3]:
c = cnx.cursor()

In [4]:
c.execute("SELECT name FROM sqlite_master WHERE TYPE = 'table';")
c.fetchall()


[('sqlite_sequence',),
 ('Player_Attributes',),
 ('Player',),
 ('Match',),
 ('League',),
 ('Country',),
 ('Team',),
 ('Team_Attributes',)]

## Finding Column Names

#### sqlite_sequence

In [5]:
pd.read_sql_query('''SELECT * FROM sqlite_sequence''', cnx).head()

,name,seq
0,Team,103916
1,Country,51958
2,League,51958
3,Match,51958
4,Player,11075


#### Player_Attributes

In [6]:
pa = pd.read_sql_query('''SELECT * FROM Player_Attributes''', cnx)
for x in pa:
    print(x)

id
player_fifa_api_id
player_api_id
date
overall_rating
potential
preferred_foot
attacking_work_rate
defensive_work_rate
crossing
finishing
heading_accuracy
short_passing
volleys
dribbling
curve
free_kick_accuracy
long_passing
ball_control
acceleration
sprint_speed
agility
reactions
balance
shot_power
jumping
stamina
strength
long_shots
aggression
interceptions
positioning
vision
penalties
marking
standing_tackle
sliding_tackle
gk_diving
gk_handling
gk_kicking
gk_positioning
gk_reflexes


#### Player

In [7]:
pd.read_sql_query('''SELECT * FROM Player''', cnx).head()

,id,player_api_id,player_name,player_fifa_api_id,birthday,height,weight
0,1,505942,Aaron Appindangoye,218353,1992-02-29 00:00:00,182.88,187
1,2,155782,Aaron Cresswell,189615,1989-12-15 00:00:00,170.18,146
2,3,162549,Aaron Doran,186170,1991-05-13 00:00:00,170.18,163
3,4,30572,Aaron Galindo,140161,1982-05-08 00:00:00,182.88,198
4,5,23780,Aaron Hughes,17725,1979-11-08 00:00:00,182.88,154


#### Match

In [17]:
match = pd.read_sql_query('''SELECT * FROM Match''', cnx).head()
for x in match:
    print(x)

id
country_id
league_id
season
stage
date
match_api_id
home_team_api_id
away_team_api_id
home_team_goal
away_team_goal
home_player_X1
home_player_X2
home_player_X3
home_player_X4
home_player_X5
home_player_X6
home_player_X7
home_player_X8
home_player_X9
home_player_X10
home_player_X11
away_player_X1
away_player_X2
away_player_X3
away_player_X4
away_player_X5
away_player_X6
away_player_X7
away_player_X8
away_player_X9
away_player_X10
away_player_X11
home_player_Y1
home_player_Y2
home_player_Y3
home_player_Y4
home_player_Y5
home_player_Y6
home_player_Y7
home_player_Y8
home_player_Y9
home_player_Y10
home_player_Y11
away_player_Y1
away_player_Y2
away_player_Y3
away_player_Y4
away_player_Y5
away_player_Y6
away_player_Y7
away_player_Y8
away_player_Y9
away_player_Y10
away_player_Y11
home_player_1
home_player_2
home_player_3
home_player_4
home_player_5
home_player_6
home_player_7
home_player_8
home_player_9
home_player_10
home_player_11
away_player_1
away_player_2
away_player_3
away_player_4
a

#### League

In [84]:
pd.read_sql_query('''SELECT * FROM League''', cnx).head()

,id,country_id,name
0,1,1,Belgium Jupiler League
1,1729,1729,England Premier League
2,4769,4769,France Ligue 1
3,7809,7809,Germany 1. Bundesliga
4,10257,10257,Italy Serie A


#### Team

In [86]:
pd.read_sql_query('''SELECT * FROM Team''', cnx).head()

,id,team_api_id,team_fifa_api_id,team_long_name,team_short_name
0,1,9987,673.0,KRC Genk,GEN
1,2,9993,675.0,Beerschot AC,BAC
2,3,10000,15005.0,SV Zulte-Waregem,ZUL
3,4,9994,2007.0,Sporting Lokeren,LOK
4,5,9984,1750.0,KSV Cercle Brugge,CEB


#### Team_Attributes

In [95]:
ta = pd.read_sql_query('''SELECT * FROM Team_Attributes''', cnx)
for x in ta:
    print(x)

id
team_fifa_api_id
team_api_id
date
buildUpPlaySpeed
buildUpPlaySpeedClass
buildUpPlayDribbling
buildUpPlayDribblingClass
buildUpPlayPassing
buildUpPlayPassingClass
buildUpPlayPositioningClass
chanceCreationPassing
chanceCreationPassingClass
chanceCreationCrossing
chanceCreationCrossingClass
chanceCreationShooting
chanceCreationShootingClass
chanceCreationPositioningClass
defencePressure
defencePressureClass
defenceAggression
defenceAggressionClass
defenceTeamWidth
defenceTeamWidthClass
defenceDefenderLineClass


### Question 1

In [15]:
pd.read_sql_query('''
    SELECT team_long_name, sum(home_team_goal) AS Total_Goals
    FROM Match
    JOIN Team
    ON Match.home_team_api_id = Team.team_api_id
    GROUP BY Match.home_team_api_id
    ORDER BY Total_Goals DESC
    LIMIT 1; 
    ''', cnx)

,team_long_name,Total_Goals
0,Real Madrid CF,505


### Question 2

In [16]:
pd.read_sql_query('''
    SELECT team_long_name, sum(away_team_goal) AS Total_Goals
    FROM Match
    JOIN Team
    ON Match.home_team_api_id = Team.team_api_id
    GROUP BY Match.home_team_api_id
    ORDER BY Total_Goals DESC
    LIMIT 1; 
    ''', cnx)

,team_long_name,Total_Goals
0,Kilmarnock,238


### Question 3

In [19]:
pd.read_sql_query('''
    SELECT COUNT(*) AS Ties
    FROM Match
    WHERE home_team_goal = away_team_goal;
    ''', cnx)

,Ties
0,6596


### Question 4

In [12]:
pd.read_sql_query('''
    SELECT COUNT(player_name) AS Total_Smiths
    FROM Player
    WHERE player_name LIKE "% smith";
    ''', cnx)

,Total_Smiths
0,15


In [13]:
pd.read_sql_query('''
    SELECT COUNT(player_name) AS [Total_*smith*s]
    FROM Player
    WHERE player_name LIKE "%smith%";
    ''', cnx)

,Total_*smith*s
0,18


### Question 5

### Question 6